In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/*'

In [3]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'B0631' in p:

        file_list.append(p)
        frame = p.split('_')[-3][:4]
        out_file.append('B0631_r'+ frame + '_cleaned.fits')

In [4]:
file_list

['../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0040-B0631+519_LRISr_2020Nov21T121539.859.fits',
 '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0041-B0631+519_LRISr_2020Nov21T132253.443.fits',
 '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0042-B0631+519_LRISr_2020Nov21T143637.210.fits']

In [5]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 560])#, transpose=True)
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)
    spec.szap_new(infile='B0631_median_image.fits')
    
    filled_data = spec.data
    mask = spec.szap_mask
    
    wav_im = pf.open(p)[19].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    sky_im = pf.open(p)[14].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    ivar_im = pf.open(p)[13].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    #print(ivar_im.shape)
    prim_hdr = pf.Header()
    prim_hdr.append(('OBJECT', 'B0631+519'))
    prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
    phdu = pf.PrimaryHDU(header=prim_hdr)
    
    sci_hdr = pf.Header()
    sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
    sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')
    
    ivar_hdr = pf.Header()
    ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
    ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')
    
    sky_hdr = pf.Header()
    sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
    sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')
    
    wav_hdr = pf.Header()
    wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
    wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')
    
    mask_hdr = pf.Header()
    mask_hdr.append(('IMTYPE', 'MASK'))
    cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'
    mask_hdr.append(('', cmt))
    mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')
    
    hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
    hdul.writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0040-B0631+519_LRISr_2020Nov21T121539.859.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0040-B0631+519_LRISr_2020Nov21T121539.859.fits (HDU=12)
The input dataset was trimmed
 xrange: 30:561.  yrange: 0:4096
Final data dimensions (x y): 531 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 531


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r2

In [ ]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 560])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)
pf.PrimaryHDU(median_im).writeto('B0631_median_image.fits')